In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import optuna
from torch.utils.data import DataLoader, TensorDataset, Dataset
from datetime import datetime
from sklearn.preprocessing import MinMaxScaler
import matplotlib.dates as mdates

In [ ]:
best_params = {}
best_value = 10
print("Best params:", study.best_params)
print("Best validation loss:", study.best_value)

if study.best_value != float('inf'):
  try:
    df_tuning = pd.read_csv('tuning.csv')
  except:
    df_tuning = pd.DataFrame(columns=['model', 'trials', 'val_loss', 'parameters'])

  new_row = {'model': args.model, 'trials': len(study.trials), 'val_loss': study.best_value, 'parameters': study.best_params}
  new_row_df = pd.DataFrame([new_row]).dropna(axis=1, how='all')
  df_tuning = pd.concat([df_tuning, new_row_df], ignore_index=True)
  df_tuning = df_tuning.sort_values(by=['model', 'val_loss'], ascending=True).reset_index(drop=True)
  df_tuning.to_csv('tuning.csv', index=False)


In [20]:
df = pd.read_csv('ColoradoData_cleaned.csv')
df = df.loc[df['Station_Name'] == 'BOULDER / JUNCTION ST1']
df.index = df['Start_DateTime']
df = df[['Start_DateTime', 'Energy_Consumption']].sort_index()
df.dropna(inplace=True)
df['Start_DateTime'] = pd.to_datetime(df['Start_DateTime'], format='%Y-%m-%d %H:%M:%S')
df.set_index('Start_DateTime', inplace=True)
print(df)

                     Energy_Consumption
Start_DateTime                         
2018-01-01 17:49:00               6.504
2018-01-01 17:49:00               6.504
2018-01-02 08:52:00               2.481
2018-01-02 08:52:00               2.481
2018-01-02 21:11:00              15.046
...                                 ...
2019-08-06 08:48:00               0.000
2019-08-07 08:56:00               0.000
2019-08-07 08:56:00               0.000
2019-08-07 10:17:00              11.060
2019-08-07 10:17:00              11.060

[968 rows x 1 columns]


In [ ]:
def create_inout_sequences(input_data, tw):
    inout_seq = []
    L = len(input_data)
    for i in range(L-tw):
        train_seq = input_data[i:i+tw]
        train_label = input_data[i+tw:i+tw+1]
        inout_seq.append((train_seq ,train_label))
    return inout_seq

all_data = df['Energy_Consumption'].values.astype(float)
train_size = int(0.6 * len(df))
train_set = all_data[:train_size]
scaler = MinMaxScaler(feature_range=(-1, 1))
train_data_normalized = scaler.fit_transform(train_set.reshape(-1, 1))
train_data_normalized = torch.FloatTensor(train_data_normalized).view(-1)

## Define Model

In [21]:
class LSTM(nn.Module):
    def __init__(self, input_size=1, hidden_layer_size=100, output_size=1):
        super().__init__()
        self.hidden_layer_size = hidden_layer_size
        self.lstm = nn.LSTM(input_size, hidden_layer_size)
        self.linear = nn.Linear(hidden_layer_size, output_size)
        self.hidden_cell = (torch.zeros(1,1,self.hidden_layer_size), torch.zeros(1,1,self.hidden_layer_size))

    def forward(self, input_seq):
        lstm_out, self.hidden_cell = self.lstm(input_seq.view(len(input_seq) ,1, -1), self.hidden_cell)
        predictions = self.linear(lstm_out.view(len(input_seq), -1))
        return predictions[-1]

## Define Objective Function

In [27]:
def objective(trial):
    # Hyperparameters to tune
    hidden_size = trial.suggest_int('hidden_size', 128, 512)
    learning_rate = trial.suggest_float('lr', 1e-4, 1e-1, log=True)
    train_window = trial.suggest_int('train_window', 12, 160)
    epochs = 150

    train_inout_seq = create_inout_sequences(train_data_normalized, train_window)
    model = LSTM()
    loss_function = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

    model.train()

    for i in range(epochs):
        for seq, labels in train_inout_seq:
            optimizer.zero_grad()
            model.hidden_cell = (torch.zeros(1, 1, model.hidden_layer_size), torch.zeros(1, 1, model.hidden_layer_size))
            y_pred = model(seq)
            loss = loss_function(y_pred, labels)
            loss.backward()
            optimizer.step()
    return loss.item()

In [28]:
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=100)
print("Best Hyperparameters:", study.best_params)

[I 2025-03-12 11:17:14,773] A new study created in memory with name: no-name-7ade5132-4c22-4a62-ad6e-465aa7e2b9c0
[I 2025-03-12 11:20:51,471] Trial 0 finished with value: 0.00086731999181211 and parameters: {'hidden_size': 274, 'lr': 0.03899978305560271, 'train_window': 133}. Best is trial 0 with value: 0.00086731999181211.
[I 2025-03-12 11:24:30,454] Trial 1 finished with value: 0.0025709925685077906 and parameters: {'hidden_size': 304, 'lr': 0.07321431399338092, 'train_window': 140}. Best is trial 0 with value: 0.00086731999181211.
[I 2025-03-12 11:27:38,385] Trial 2 finished with value: 0.0061284820549190044 and parameters: {'hidden_size': 442, 'lr': 0.013649126985391178, 'train_window': 60}. Best is trial 0 with value: 0.00086731999181211.
[I 2025-03-12 11:30:20,264] Trial 3 finished with value: 0.0016303230077028275 and parameters: {'hidden_size': 314, 'lr': 0.00016044574466476645, 'train_window': 52}. Best is trial 0 with value: 0.00086731999181211.
[I 2025-03-12 11:32:49,947] Tr

KeyboardInterrupt: 